In [1]:
import cv2
import matplotlib.pyplot as plt

import xml.etree.ElementTree as ET
from glob import glob
from tqdm import tqdm
import shutil
import os

import multiprocessing
from multiprocessing.dummy import Pool


%matplotlib inline

# 图片展示函数
def show(img):
    plt.imshow(img[:,:,::-1])

In [2]:
newpath = 'abnormal_with_xml' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
    
folder_names = ['../xuelang_round1_train_part1_20180628',
               '../xuelang_round1_train_part2_20180705',
               '../xuelang_round1_train_part3_20180709']

for folder_name in folder_names:
    with tqdm(glob(f'{folder_name}/*/*.jpg')) as pbar:
        for fname in pbar:
            if '正常' in fname:
                continue
            else:
                xml_fname = fname.replace('.jpg', '.xml')
                shutil.copy(fname, 'abnormal_with_xml' )
                shutil.copy(xml_fname, 'abnormal_with_xml' )

100%|██████████| 615/615 [00:01<00:00, 405.14it/s]


In [4]:
def crop(fname):
    size = 256
    half_size = size/2
    xml_fname = fname.replace('.jpg', '.xml')
    tree = ET.parse(xml_fname)
    img_fname = fname[fname.rfind('/')+1:].replace('.jpg', '')
    img = cv2.imread(fname)
    height, width, _ = img.shape
    for i, ob in enumerate(tree.findall('object')):
        bbox = ob.find('bndbox')
        tagname = ob.find('name').text
        x1, y1, x2, y2 = [int(x.text) for x in bbox.getchildren()]
        center_x = (x2 - x1)/2 + x1
        center_y = (y2 - y1)/2 + y1
    
        new_x1 = int(center_x - half_size)
        new_x2 = int(center_x + half_size)
        new_y1 = int(center_y - half_size)
        new_y2 = int(center_y + half_size)
        
        if new_x1 < 0:
            new_x1 = 0
            new_x2 = size
        
        if new_x2 > width:
            new_x1 = width - size
            new_x2 = width
        
        if new_y1 < 0:
            new_y1 = 0
            new_y2 = size
        
        if new_y2 > height:
            new_y1 = height - size
            new_y2 = height
        
        crop_img = img[new_y1:new_y2, new_x1:new_x2]
        if crop_img.shape[0] != 256 or crop_img.shape[1] != 256:
            print(fname, crop_img.shape, new_x1, new_y1, new_x2, new_y2)
        cv2.imwrite(f'abnormal_crop/{img_fname}_{i}.png', crop_img)

In [5]:
fname_test = 'abnormal_with_xml/J01_2018.06.19 15_45_28.jpg'
crop(fname_test)

In [7]:
!rm -rf abnormal_crop
!mkdir abnormal_crop

In [8]:
fnames = glob('abnormal_with_xml/*.jpg')
with Pool(multiprocessing.cpu_count()) as pool:
    with tqdm(pool.imap_unordered(crop, fnames), total=len(fnames)) as pbar:
        for _ in pbar:
            pass

100%|██████████| 706/706 [00:05<00:00, 137.30it/s]
